# encrypted-ezkl

Here's an example leveraging EZKL whereby the inputs to the model are encrypted using el-gamal encryption (then hashed). 

In this setup:
- the hashes of the enecrypted values are known to the prover and verifier
- the hashes serve as "public inputs" (a.k.a instances) to the circuit along with the 
- the encrypted values can be safely shared between the prover and verifier, without the verifier learning the unencrypted values
- We leave the outputs of the model as public as well (known to the  verifier and prover).

Summary of el gamal: 

### 1. Key generation

We use the G1 group of bn256 of order $q$ and generator $g$
Choose an integer x randomly from $\{ 1 , … , q − 1 \}$
Compute $h:=g^x$
$x$ is the private key and $h$ + group information form the public key. 

### 2. Encryption 

We can encrypt a message using the public key: 
Choose an integer y randomly from $\{ 1 , … , q − 1 \}$
Compute  ${\displaystyle s:=h^{y}}$. This is called the shared secret.
Compute ${\displaystyle c_{1}:=g^{y}}$.
Compute ${\displaystyle c_{2}:=m + poseidon(s.x, s.y)}$. 
We use `poseidon` hashing to reduce the (effectively) two dimensional coordinates of $s$ ($s.x$, $s.y$) in G1 to a single field element  and we also go from G1Affine to the native field of the SNARK `Fr`(scalar field of G1), as the message is also in `Fr`

The ciphertext is the pair $(c_1, c_2)$. 

### 3. Decryption 

Decryption then is as follows: 

Compute ${\displaystyle s:=c_{1}^{x}}$. Since ${\displaystyle c_{1}=g^{y}}$, ${\displaystyle c_{1}^{x}=g^{xy}=h^{y}}$. 
Now Compute ${\displaystyle m:=c_2 - poseidon(s.x, s.y)}$.

**Other perspective**:  s, is as a [one-time pad](https://en.wikipedia.org/wiki/One-time_pad) for encrypting the message

### 4. Elements of the Circuit

1. The private key x is publicly committed using a poseidon hash. The circuit verifies that the prover knows a private key that corresponds to this publicly committed to hash. This assumes that the prover has generated both the public and private keys and is planning to send the decryption key to another user. 

2. Verifies that given 
 - public inputs $poseidon(c_2), c_1, poseidon(x)$
 - private inputs $m, x$
-> $x$ is indeed the decryption key
->  m encrypted using $s$ is indeed $c_2$ 
-> $c_1$ was properly derived




First we import the necessary dependencies and set up logging to be as informative as possible. 

In [2]:
from torch import nn
import ezkl
import os
import json
import logging

# uncomment for more descriptive logging 
FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.DEBUG)


Now we define our model. It is a humble model with but a conv layer and a $ReLU$ non-linearity, but it is a model nonetheless

In [3]:
import torch
# Defines the model
# we got convs, we got relu, 
# What else could one want ????

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=1, kernel_size=5, stride=4)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)

        return x


circuit = MyModel()

# this is where you'd train your model




We omit training for purposes of this demonstration. We've marked where training would happen in the cell above. 
Now we export the model to onnx and create a corresponding (randomly generated) input file.

You can replace the random `x` with real data if you so wish. 

In [4]:
x = 0.1*torch.rand(1,*[3, 8, 8], requires_grad=True)

# Flips the neural net into inference mode
circuit.eval()

    # Export the model
torch.onnx.export(circuit,               # model being run
                      x,                   # model input (or a tuple for multiple inputs)
                      "network.onnx",            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

    # Serialize data into file:
json.dump( data, open("input.json", 'w' ))



================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



This is where the magic happens. We define our `PyRunArgs` objects which contains the visibility parameters for out model. 
- `input_visibility` defines the visibility of the model inputs
- `param_visibility` defines the visibility of the model weights and constants and parameters 
- `output_visibility` defines the visibility of the model outputs

There are currently 4 visibility settings:
- `public`: known to both the verifier and prover (a subtle nuance is that this may not be the case for model parameters but until we have more rigorous theoretical results we don't want to make strong claims as to this). 
- `private`: known only to the prover
- `hashed`: the hash pre-image is known to the prover, the prover and verifier know the hash. The prover proves that the they know the pre-image to the hash. 
- `encrypted`: the non-encrypted element and the secret key used for decryption are known to the prover. The prover and the verifier know the encrypted element, the public key used to encrypt, and the hash of the decryption hey. The prover proves that they know the pre-image of the hashed decryption key and that this key can in fact decrypt the encrypted message.

Here we create the following setup:
- `input_visibility`: "encrypted"
- `param_visibility`: "public"
- `output_visibility`: public

We encourage you to play around with other setups :) 

Shoutouts: 

- [summa-solvency](https://github.com/summa-dev/summa-solvency) for their help with the poseidon hashing chip. 
- [timeofey](https://github.com/timoftime) for providing inspiration in our developement of the el-gamal encryption circuit in Halo2. 

In [5]:
import ezkl

model_path = os.path.join('network.onnx')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')
srs_path = os.path.join('kzg.srs')
data_path = os.path.join('input.json')

run_args = ezkl.PyRunArgs()
run_args.input_visibility = "encrypted"
run_args.param_visibility = "public"
run_args.output_visibility = "public"





Now we generate a settings file. This file basically instantiates a bunch of parameters that determine their circuit shape, size etc... Because of the way we represent nonlinearities in the circuit (using Halo2's [lookup tables](https://zcash.github.io/halo2/design/proving-system/lookup.html)), it is often best to _calibrate_ this settings file as some data can fall out of range of these lookups.

You can pass a dataset for calibration that will be representative of real inputs you might find if and when you deploy the prover. Here we create a dummy calibration dataset for demonstration purposes. 

In [6]:
!RUST_LOG=trace
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path, py_run_args=run_args)
assert res == True

DEBUG tract_onnx.model 2023-07-20 16:05:27,345 model.rs:247 ONNX operator set version: 10
DEBUG tract_hir.infer.analyser 2023-07-20 16:05:27,346 analyser.rs:151   Refined 3/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_hir.infer.analyser 2023-07-20 16:05:27,346 analyser.rs:151   Refined 4/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_core.optim.change_axes 2023-07-20 16:05:27,346 change_axes.rs:76   Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-20 16:05:27,347 change_axes.rs:88     Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2023-07-20 16:05:27,347 change_axes.rs:76   Considering change AxisChange { outlet: 1/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-20 16:05:27,348 change_axes.rs:88     Change AxisChange { outlet: 1/0>, op: Rm(0) } blocked by locked interface 3/0>
DEBUG tract_core.optim.change_axes 2023-07-20 16:05:27,348 change_axes.rs:76   Considering change Axis

In [12]:
# generate a bunch of dummy calibration data
cal_data = {
    "input_data": [(0.1*torch.rand(20, *[3, 8, 8])).flatten().tolist()],
}

cal_path = os.path.join('val_data.json')
# save as json file
with open(cal_path, "w") as f:
    json.dump(cal_data, f)

res = await ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources")

DEBUG tract_onnx.model 2023-07-20 16:08:10,153 model.rs:247 ONNX operator set version: 10
DEBUG tract_hir.infer.analyser 2023-07-20 16:08:10,154 analyser.rs:151   Refined 3/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_hir.infer.analyser 2023-07-20 16:08:10,155 analyser.rs:151   Refined 4/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_core.optim.change_axes 2023-07-20 16:08:10,156 change_axes.rs:76   Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-20 16:08:10,157 change_axes.rs:88     Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2023-07-20 16:08:10,157 change_axes.rs:76   Considering change AxisChange { outlet: 1/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-20 16:08:10,157 change_axes.rs:88     Change AxisChange { outlet: 1/0>, op: Rm(0) } blocked by locked interface 3/0>
DEBUG tract_core.optim.change_axes 2023-07-20 16:08:10,158 change_axes.rs:76   Considering change Axis

As we use Halo2 with KZG-commitments we need an SRS string from (preferably) a multi-party trusted setup ceremony. For an overview of the procedures for such a ceremony check out [this page](https://blog.ethereum.org/2023/01/16/announcing-kzg-ceremony). The `get_srs` command retrieves a correctly sized SRS given the calibrated settings file from [here](https://github.com/han0110/halo2-kzg-srs). 

These SRS were generated with [this](https://github.com/privacy-scaling-explorations/perpetualpowersoftau) ceremony. 

In [8]:
res = ezkl.get_srs(srs_path, settings_path)


DEBUG reqwest.connect 2023-07-20 16:06:46,989 connect.rs:429 starting new connection: https://trusted-setup-halo2kzg.s3.eu-central-1.amazonaws.com/
INFO ezkl.execute 2023-07-20 16:07:31,848 execute.rs:490 SRS downloaded


We now need to generate the (partial) circuit witness. These are the model outputs (and any hashes) that are generated when feeding the previously generated `input.json` through the circuit / model. 

In [13]:
!export RUST_BACKTRACE=1

witness_path = "witness.json"

res = ezkl.gen_witness(data_path, model_path, witness_path, settings_path = settings_path)

DEBUG tract_onnx.model 2023-07-20 16:10:02,255 model.rs:247 ONNX operator set version: 10
DEBUG tract_hir.infer.analyser 2023-07-20 16:10:02,256 analyser.rs:151   Refined 3/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_hir.infer.analyser 2023-07-20 16:10:02,256 analyser.rs:151   Refined 4/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_core.optim.change_axes 2023-07-20 16:10:02,257 change_axes.rs:76   Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-20 16:10:02,257 change_axes.rs:88     Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2023-07-20 16:10:02,258 change_axes.rs:76   Considering change AxisChange { outlet: 1/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-20 16:10:02,258 change_axes.rs:88     Change AxisChange { outlet: 1/0>, op: Rm(0) } blocked by locked interface 3/0>
DEBUG tract_core.optim.change_axes 2023-07-20 16:10:02,258 change_axes.rs:76   Considering change Axis

As a sanity check you can "mock prove" (i.e check that all the constraints of the circuit match without generate a full proof). 

In [14]:


res = ezkl.mock(witness_path, model_path, settings_path)

DEBUG tract_onnx.model 2023-07-20 16:10:04,319 model.rs:247 ONNX operator set version: 10
DEBUG tract_hir.infer.analyser 2023-07-20 16:10:04,320 analyser.rs:151   Refined 3/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_hir.infer.analyser 2023-07-20 16:10:04,321 analyser.rs:151   Refined 4/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_core.optim.change_axes 2023-07-20 16:10:04,321 change_axes.rs:76   Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-20 16:10:04,322 change_axes.rs:88     Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2023-07-20 16:10:04,322 change_axes.rs:76   Considering change AxisChange { outlet: 1/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-20 16:10:04,323 change_axes.rs:88     Change AxisChange { outlet: 1/0>, op: Rm(0) } blocked by locked interface 3/0>
DEBUG tract_core.optim.change_axes 2023-07-20 16:10:04,323 change_axes.rs:76   Considering change Axis

Here we setup verifying and proving keys for the circuit. As the name suggests the proving key is needed for ... proving and the verifying key is needed for ... verifying. 

In [15]:
# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK
res = ezkl.setup(
        model_path,
        vk_path,
        pk_path,
        srs_path,
        settings_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

DEBUG tract_onnx.model 2023-07-20 16:10:26,195 model.rs:247 ONNX operator set version: 10
DEBUG tract_hir.infer.analyser 2023-07-20 16:10:26,196 analyser.rs:151   Refined 3/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_hir.infer.analyser 2023-07-20 16:10:26,196 analyser.rs:151   Refined 4/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_core.optim.change_axes 2023-07-20 16:10:26,197 change_axes.rs:76   Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-20 16:10:26,197 change_axes.rs:88     Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2023-07-20 16:10:26,197 change_axes.rs:76   Considering change AxisChange { outlet: 1/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-20 16:10:26,198 change_axes.rs:88     Change AxisChange { outlet: 1/0>, op: Rm(0) } blocked by locked interface 3/0>
DEBUG tract_core.optim.change_axes 2023-07-20 16:10:26,198 change_axes.rs:76   Considering change Axis

Now we generate a full proof. 

In [17]:
# GENERATE A PROOF

proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        model_path,
        pk_path,
        proof_path,
        srs_path,
        "evm",
        "single",
        settings_path,
    )

print(res)
assert os.path.isfile(proof_path)

DEBUG tract_onnx.model 2023-07-20 16:11:48,121 model.rs:247 ONNX operator set version: 10
DEBUG tract_hir.infer.analyser 2023-07-20 16:11:48,122 analyser.rs:151   Refined 3/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_hir.infer.analyser 2023-07-20 16:11:48,123 analyser.rs:151   Refined 4/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_core.optim.change_axes 2023-07-20 16:11:48,123 change_axes.rs:76   Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-20 16:11:48,124 change_axes.rs:88     Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2023-07-20 16:11:48,124 change_axes.rs:76   Considering change AxisChange { outlet: 1/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-20 16:11:48,124 change_axes.rs:88     Change AxisChange { outlet: 1/0>, op: Rm(0) } blocked by locked interface 3/0>
DEBUG tract_core.optim.change_axes 2023-07-20 16:11:48,124 change_axes.rs:76   Considering change Axis

{'instances': [[[0, 0, 0, 0]], [[13668030090413849173, 1960211732490268162, 311977267056297772, 3128979392541613496], [12766697592520233576, 10935220065613224600, 11132480334689525590, 3074479748062098777], [10495669562412616489, 742326491355375407, 6071634436851262108, 418301285806554622], [6080934496034518721, 5843501826335168448, 13489856285899845251, 2999042662606665795]]], 'proof': '0442b752aae041e227883eda93ea12a1551f061602ac0de0f2c94986e71236ab2bcc48879d800fe4781d39641b85d9764968d0956ddeffd908597a63880f545821dfc8dd496a65791621f779a2a319d33d4c3de64c57a7dbfcc7cd5736b3159e0a69b79b2c16585b08018ff4b5c6437ec03101b0a1c6c87bf0a973802131968d15cae54af71a9e124023ded4db3e417e7c07e7e35b5edccec3028e20f079b8291fdb831250c4b3c06b4cb939d0c403be54d4f20f32d2d0a4eed9406a7166e595020604aedc04713fa0a61613bebb8674ff63d01d098960696684b5fe359fee65233564fe032e4a37acb8d0b984650121349525379f69a6d434b0b52d561c9e1707859d55c0580c16f11c3508fd51c6d6cca4d1f629117b19f3a2011271de327f23353b0aacdf6a1e1f1cdb556920b08a5

And verify it as a sanity check. 

In [18]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        srs_path,
    )

assert res == True
print("verified")

INFO ezkl.pfsys.srs 2023-07-20 16:14:26,370 srs.rs:23 loading srs from "kzg.srs"
INFO ezkl.execute 2023-07-20 16:14:26,425 execute.rs:1599 downsizing params to 19 logrows
INFO ezkl.pfsys 2023-07-20 16:14:26,426 mod.rs:508 loading verification key from "test.vk"
INFO ezkl.graph.model 2023-07-20 16:14:26,427 model.rs:572 configuring model
DEBUG ezkl.circuit.ops.chip 2023-07-20 16:14:26,427 chip.rs:341 assigning lookup input
DEBUG ezkl.circuit.ops.chip 2023-07-20 16:14:26,428 chip.rs:345 assigning lookup output
INFO ezkl.execute 2023-07-20 16:14:26,492 execute.rs:1558 verify took 0.4
INFO ezkl.execute 2023-07-20 16:14:26,493 execute.rs:1563 verified: true


verified


We can now create an EVM / `.sol` verifier that can be deployed on chain to verify submitted proofs using a view function.

In [19]:

abi_path = 'test.abi'
sol_code_path = 'test.sol'

res = ezkl.create_evm_verifier(
        vk_path,
        srs_path,
        settings_path,
        sol_code_path,
        abi_path,
    )
assert res == True


INFO ezkl.execute 2023-07-20 16:14:30,592 execute.rs:80 checking solc installation..
DEBUG ezkl.execute 2023-07-20 16:14:30,678 execute.rs:85 solc output: Output {
    status: ExitStatus(
        unix_wait_status(
            0,
        ),
    ),
    stdout: "solc, the solidity compiler commandline interface\nVersion: 0.8.20+commit.a1b79de6.Darwin.appleclang\n",
    stderr: "",
}
DEBUG ezkl.execute 2023-07-20 16:14:30,678 execute.rs:87 solc output success: true
DEBUG ezkl.execute 2023-07-20 16:14:30,678 execute.rs:90 solc check passed, proceeding
INFO ezkl.pfsys.srs 2023-07-20 16:14:30,679 srs.rs:23 loading srs from "kzg.srs"
INFO ezkl.execute 2023-07-20 16:14:30,717 execute.rs:1599 downsizing params to 19 logrows
INFO ezkl.pfsys 2023-07-20 16:14:30,718 mod.rs:508 loading verification key from "test.vk"
INFO ezkl.graph.model 2023-07-20 16:14:30,718 model.rs:572 configuring model
DEBUG ezkl.circuit.ops.chip 2023-07-20 16:14:30,718 chip.rs:341 assigning lookup input
DEBUG ezkl.circuit.op

## Verify on the evm

If we use the simple verifier, the encryption components of the circuit push the size of the verifier above the 24kb limit of the EVM. We demonstrate this by using the simple verifier and then get an on-chain verifier by using `aggregation`. 

In [20]:
# Make sure anvil is running locally first
# run with $ anvil -p 3030
# we use the default anvil node here
import json

address_path = os.path.join("address.json")

try:
    ezkl.deploy_evm(
    address_path,
    sol_code_path,
    'http://127.0.0.1:3030')
except BaseException as e:
    print(e)



INFO ezkl.execute 2023-07-20 16:14:34,153 execute.rs:80 checking solc installation..
DEBUG reqwest.connect 2023-07-20 16:14:34,196 connect.rs:429 starting new connection: http://127.0.0.1:3030/
INFO ezkl.eth 2023-07-20 16:14:34,197 eth.rs:77 using chain 31337
DEBUG ezkl.eth 2023-07-20 16:14:34,559 eth.rs:537 runtime bytecode size: 31145


Solidity runtime bytecode size is: 31145,
            which exceeds 24577 bytes limit.


thread '<unnamed>' panicked at 'Solidity runtime bytecode size is: 31145,
            which exceeds 24577 bytes limit.', src/eth.rs:540:9


We now need to recreate the proof in a format that is aggregatable. This means that we use a "poseidon" transcript (simpler to aggregate over) and we use an "accum" stragetgy for the prover.

In [28]:
# fetch the SRS for 20 logrows
with open(settings_path, "r") as f:
    settings = json.load(f)
    settings["run_args"]["logrows"] = 22

new_settings_path = os.path.join("new_settings.json")
new_srs_path = os.path.join("new_kzg.srs")
# now save
with open(new_settings_path, "w") as f:    
    json.dump(settings, f)
#  get srs
res = ezkl.get_srs(new_srs_path, new_settings_path)

DEBUG reqwest.connect 2023-07-20 16:22:33,416 connect.rs:429 starting new connection: https://trusted-setup-halo2kzg.s3.eu-central-1.amazonaws.com/
INFO ezkl.execute 2023-07-20 16:29:51,077 execute.rs:490 SRS downloaded


In [22]:


res = ezkl.prove(
        witness_path,
        model_path,
        pk_path,
        proof_path,
        srs_path,
        "poseidon",
        "accum",
        settings_path,
    )

DEBUG tract_onnx.model 2023-07-20 16:16:43,231 model.rs:247 ONNX operator set version: 10
DEBUG tract_hir.infer.analyser 2023-07-20 16:16:43,232 analyser.rs:151   Refined 3/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_hir.infer.analyser 2023-07-20 16:16:43,233 analyser.rs:151   Refined 4/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_core.optim.change_axes 2023-07-20 16:16:43,234 change_axes.rs:76   Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-20 16:16:43,234 change_axes.rs:88     Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2023-07-20 16:16:43,234 change_axes.rs:76   Considering change AxisChange { outlet: 1/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-20 16:16:43,235 change_axes.rs:88     Change AxisChange { outlet: 1/0>, op: Rm(0) } blocked by locked interface 3/0>
DEBUG tract_core.optim.change_axes 2023-07-20 16:16:43,235 change_axes.rs:76   Considering change Axis

In [1]:

aggregate_proof_path = os.path.join('aggr_evm.pf')
aggregate_vk_path = os.path.join('aggr_evm.vk')



res = ezkl.aggregate(
        aggregate_proof_path,
        [proof_path],
        aggregate_vk_path,
        new_srs_path,
        "evm",
        22,
        "unsafe"
    )

assert res == True
assert os.path.isfile(aggregate_proof_path)
assert os.path.isfile(aggregate_vk_path)

In [ ]:
# we now create the evm contract for verifying the aggregated proof

sol_code_path = os.path.join(folder_path, 'aggr_evm.sol')
abi_path = os.path.join(folder_path, 'aggr_evm.abi')

res = ezkl.create_evm_verifier_aggr(
        aggregate_vk_path,
        new_srs_path,
        sol_code_path,
        abi_path,
        [settings_path]
    )


In [ ]:
# Make sure anvil is running locally first
# run with $ anvil -p 3030
# we use the default anvil node here

address_path = os.path.join("address_aggr.json")

ezkl.deploy_evm(
    address_path,
    sol_code_path,
    'http://127.0.0.1:3030')

assert res == True

with open(address_path, 'r') as file:
    addr = file.read().rstrip()

In [ ]:
# make sure anvil is running locally
# $ anvil -p 3030

res = ezkl.verify_evm(
    proof_path,
    addr,
    "http://127.0.0.1:3030"
)
assert res == True